In [1]:
#@title 檢視 GPU
! nvidia-smi

Sat Mar 19 00:27:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title 安裝 colorama, ffmpeg

! pip install colorama
! pip install ffmpeg-python

In [ ]:
#@title 安裝 DeepFaceLab
!git clone https://github.com/iperov/DeepFaceLab.git

In [4]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#@title 防止 Google Colab 斷開連線

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [7]:
#@title 將原素材從 Google Drive 搬回 VM
workspace_file = '/content/drive/MyDrive/aidataset/deepfacelab/deepfacelab_ws2.zip' #@param {type:"string"}
cmd  = f'cp {workspace_file} ./'
! $cmd

In [ ]:
#@title 解壓縮原素材Workspace
zip_file = 'deepfacelab_ws2.zip' #@param {type:"string"}
cmd  = f'unzip  {zip_file}'
! $cmd

In [13]:
#@title 移除 data_dst 既有素材

! rm -f /content/workspace/data_dst.mp4
! rm -rf /content/workspace/data_dst/*

#如果要處理新的影片

In [14]:
#@title 從 Google Drive 複製新data_dst影片到workspace

dst_mp4 = '/content/drive/MyDrive/aidataset/deepfacelab/data_dst_xseg.mp4' #@param {type:"string"}
dst_cmd=f"cp {dst_mp4} /content/workspace/data_dst.mp4" 
! $dst_cmd

In [ ]:
#@title 從新 data_dst 影片擷取人物圖片
# 開始擷取 dst
%cd /content/DeepFaceLab/
! python main.py videoed extract-video --input-file /content/workspace/data_dst.mp4 --output-dir /content/workspace/data_dst/

In [ ]:
#@title 新 data_dst 影片降噪 (選擇性)
%cd /content/DeepFaceLab/
! python  main.py videoed denoise-image-sequence --input-dir /content/workspace/data_dst 

In [ ]:
#@title 校正新 data_dst 影片
%cd /content/DeepFaceLab/
! python main.py extract --input-dir /content/workspace/data_dst --output-dir /content/workspace/data_dst/aligned --detector s3fd

In [ ]:
#@title 排序 data_dst 影格 (選擇性)
%cd /content/DeepFaceLab/
! python main.py sort --input-dir /content/workspace/data_dst/aligned --by hist

In [ ]:
#@title 打包 data_dst 影格
%cd /content/DeepFaceLab/
!python main.py util --input-dir /content/workspace/data_dst/aligned  --pack-faceset

In [ ]:
#@title 壓縮素材
% cd /content/
workspace = 'deepfacelab_ws_new' #@param {type:"string"}
cmd = f'zip -r {workspace} workspace/*'
! $cmd

In [ ]:
#@title 將素材搬移至 Google Drive
workspace_dir = '/content/drive/MyDrive/aidataset/deepfacelab/' #@param {type:"string"}
cmd  = f'cp {workspace}.zip {workspace_dir}'
! $cmd

#如果已經整理好新的 Workspace

In [ ]:
#@title 替換 data_dst 素材
zip_file = 'deepfacelab_ws_new.zip' #@param {type:"string"}

%cd /content/
! mkdir -p tmp
cmd =  f'cp /content/drive/MyDrive/aidataset/deepfacelab/{zip_file}  tmp/'
! $cmd
cmd =  f'cd tmp; unzip {zip_file}'
! $cmd

! rm /content/workspace/data_dst/*
! rm /content/workspace/data_dst.mp4
! rm /content/workspace/data_dst/aligned/*
! cp /content/tmp/workspace/data_dst/*.png workspace/data_dst/
! cp /content/tmp/workspace/data_dst/aligned/* workspace/data_dst/aligned/
! cp /content/tmp/workspace/data_dst.mp4 /content/workspace/

In [ ]:
A#@title 打包 data_dst 影格
%cd /content/DeepFaceLab/
!python main.py util --input-dir /content/workspace/data_dst/aligned  --pack-faceset

# 影像合併

In [ ]:
#@title 合併影格
%cd /content/DeepFaceLab/
model_path = '/content/drive/MyDrive/aidataset/deepfacelab/model_20220312/' #@param {type:"string"}
cmd = f'python main.py merge --input-dir /content/workspace/data_dst --output-dir /content/workspace/data_dst/merged --output-mask-dir /content/workspace/data_dst/merged_mask --aligned-dir /content/workspace/data_dst/aligned --model-dir {model_path} --model SAEHD'
! $cmd

In [ ]:
#@title 影片生成
%cd /content/DeepFaceLab/
output_mp4 = '/content/workspace/result17.mp4' #@param {type:"string"}
cmd = f'python main.py videoed video-from-sequence --input-dir /content/workspace/data_dst/merged --output-file {output_mp4} --reference-file /content/workspace/data_dst.mp4 --include-audio'
! $cmd

In [ ]:
#@title 將影片搬至Google Drive
gd_path = '/content/drive/MyDrive/aidataset/' #@param {type:"string"}
cmd = f'cp {output_mp4} {gd_path}'
! $cmd